In [136]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [137]:
from classes.grid import Grid
from queue import PriorityQueue
import copy
import math

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = puzzle

# Part 1

In [138]:
# This is a pathfinding problem, so a good algorithm to use would be A*

class Maze(Grid):
    def __init__(self, grid):
        super().__init__(grid)

        # Extract start and end and place reindeer at the start, facing east
        self.start = self.get_tiles('S').pop()
        self.end = self.get_tiles('E').pop()
        self.walls = self.get_tiles('#')
        self.starting_dir = '>'


    def dist_to_end(self, row, col):
        '''
        Returns the 'taxicab' distance from (row,col) to the end tile
        '''
        return abs(self.reindeer_pos[0] - row) + abs(self.reindeer_pos[1] - col)

    def eval_cost(self, path):
        '''
        Evaluates the cost of the given path (+1 for moving forward, +1000 for turning 90 degrees).
        Input is of form e.g. >>>^^^^^>>vv>>><<<^^>>>

        Assumes the path is valid
        '''
        dir = self.starting_dir
        
        score = 0
        for move in path:
            if move != dir:
                score += 1000
            score += 1
            dir = move
        
        return score
    
    def draw_path(self,path):
        '''
        Returns the output of printing the grid but with a given path added on top
        '''
        orig_grid= copy.deepcopy(self.grid)
        
        pos = [self.start[0],self.start[1]]

        for move in path:
            new_pos = pos
            match move:
                case '^':
                    new_pos[0] -= 1
                case 'v':
                    new_pos[0] += 1
                case '<':
                    new_pos[1] -= 1
                case '>':
                    new_pos[1] += 1
            self.grid[new_pos[0]][new_pos[1]] = move
            pos = new_pos
        
        self.print_grid()
        self.grid = orig_grid



    def find_lowest_scoring_path(self):
        '''
        Finds the lowest scoring path from S to E in the grid
        '''
        # We want to implement A* here, from S->E with #s as walls
        # A node in this algorithm is a combination of (a tile that is not a wall) and (a direction).

        seen = set()

        # Implement a priority queue of nodes to visit next so we always go from the one with the lowest cost
        # Entries on the priority queue also include the travelling cost (score) so we can pop in order

        nodes_to_visit = PriorityQueue()
        nodes_to_visit.put((0, (self.start, self.starting_dir, '')))

        # Iterate through all nodes we have not yet explored
        while nodes_to_visit:
            cur_node = nodes_to_visit.get()
            cur_node_coords = cur_node[1][0]
            cur_node_dir = cur_node[1][1]
            cur_node_path = cur_node[1][2]
            
            # Reached the end?
            if cur_node_coords== self.end:
                return cur_node_path
            
            
            # Not at the end yet - add neighbours to exploring queue
            # 3 possible neighbours - step forward/turn right/turn left

            neighbours = set()

            # Turning left/right
            match cur_node_dir:
                case '^' | 'v':
                    new_dir_1 = '<'
                    new_dir_2 = '>'
                case '<' | '>':
                    new_dir_1 = '^'
                    new_dir_2 = 'v'

            # Skip adding if we will end up facing a wall
            turning_1_tile_in_front = self.get_relative(cur_node_coords[0],cur_node_coords[1],new_dir_1)[2]
            turning_2_tile_in_front = self.get_relative(cur_node_coords[0],cur_node_coords[1],new_dir_2)[2]

            if turning_1_tile_in_front != '#':
                neighbours.add(((cur_node_coords),new_dir_1))
            
            if turning_2_tile_in_front != '#':
                neighbours.add(((cur_node_coords),new_dir_2))

            # For stepping forward, need to make sure that we don't step into a wall
            step_forward_tile = self.get_relative(cur_node_coords[0],cur_node_coords[1],cur_node_dir)

            if step_forward_tile[2] != '#':
                neighbours.add(((step_forward_tile[0],step_forward_tile[1]), cur_node_dir))

            #print(neighbours)
            
            # Iterate through valid neighbours
            for neighbour in neighbours:
                # Skip nodes we've already visited
                if neighbour in seen:
                    continue

                # Find cost to travel to this node (cost to where the neighbour came from + cost to move to this neighbour)
                cost = self.eval_cost(cur_node_path)
                new_path = cur_node_path
                if cur_node_dir == neighbour[1]:
                    cost += 1
                    new_path += cur_node_dir
                else:
                    cost += 1000
                
                nodes_to_visit.put((cost,(neighbour[0], neighbour[1], new_path)))

                # Add neighbour to seen
                seen.add(neighbour)

        

In [139]:
maze_map = input.split('\n')

maze = Maze(maze_map)
#maze.print_grid()

best_path = maze.find_lowest_scoring_path()
#maze.draw_path(best_path)
maze.eval_cost(best_path)

122492

# Part 2

In [141]:
# Part 2 - need to tweak the algorithm to keep going once we find one optimal path until all paths left are of a higher score,
# and along the way store all of the optimal paths that we come across

class Maze(Grid):
    def __init__(self, grid):
        super().__init__(grid)

        # Extract start and end and place reindeer at the start, facing east
        self.start = self.get_tiles('S').pop()
        self.end = self.get_tiles('E').pop()
        self.walls = self.get_tiles('#')
        self.starting_dir = '>'


    def dist_to_end(self, row, col):
        '''
        Returns the 'taxicab' distance from (row,col) to the end tile
        '''
        return abs(self.reindeer_pos[0] - row) + abs(self.reindeer_pos[1] - col)

    def eval_cost(self, path):
        '''
        Evaluates the cost of the given path (+1 for moving forward, +1000 for turning 90 degrees).
        Input is of form e.g. >>>^^^^^>>vv>>><<<^^>>>

        Assumes the path is valid
        '''
        dir = self.starting_dir
        
        score = 0
        for move in path:
            if move != dir:
                score += 1000
            score += 1
            dir = move
        
        return score
    
    def draw_path(self,path, return_set_occupied=False):
        '''
        Returns the output of printing the grid but with a given path added on top
        '''
        orig_grid= copy.deepcopy(self.grid)
        
        pos = [self.start[0],self.start[1]]
        prev_dir = path[0]

        for i,move in enumerate(path):
            if i == 1:
                continue
            new_pos = pos
            if prev_dir == move:
                match move:
                    case '^':
                        new_pos[0] -= 1
                    case 'v':
                        new_pos[0] += 1
                    case '<':
                        new_pos[1] -= 1
                    case '>':
                        new_pos[1] += 1
                self.grid[new_pos[0]][new_pos[1]] = move
                pos = new_pos
            prev_dir = move
        
        if return_set_occupied:
            occupied = self.get_tiles('S').union(self.get_tiles('E')).union(self.get_tiles('<')).union(self.get_tiles('>')).union(self.get_tiles('^')).union(self.get_tiles('v'))
            self.grid = orig_grid
            return occupied
        
        self.print_grid()
        


        self.grid = orig_grid



    def find_lowest_scoring_path(self):
        '''
        Finds the lowest scoring path from S to E in the grid
        '''
        # Part 2 - lesson learned, A* can't really find all the paths
        # switching to Dijkstra's Algorithm instead...
        routes = set()

        costs = {}
        prevs = {}

        nodes_to_visit = PriorityQueue()
        nodes_to_visit.put((0, (self.start, self.starting_dir)))

        free_tiles = self.get_tiles('.').union(self.get_tiles('E')).union(self.get_tiles('S'))
        
        
        for tile in free_tiles:
            for dir in ['^','v','<','>']:
                costs[(tile,dir)] = math.inf
                prevs[(tile,dir)] = None
                nodes_to_visit.put((math.inf, (tile,dir)))
        
        costs[(self.start, self.starting_dir)] = 0
        
        #i=0
        while nodes_to_visit:
            if nodes_to_visit.qsize() == 0:
                break
            # i += 1
            # if i % 1000 == 0:
            #     print(nodes_to_visit.qsize())


            cur_node = nodes_to_visit.get()

            #Find node neighbours
            cur_node_coords = cur_node[1][0]
            cur_node_dir = cur_node[1][1]

            neighbours = set()

            # Turning left/right
            match cur_node_dir:
                case '^' | 'v':
                    new_dir_1 = '<'
                    new_dir_2 = '>'
                case '<' | '>':
                    new_dir_1 = '^'
                    new_dir_2 = 'v'

            # Skip adding if we will end up facing a wall
            turning_1_tile_in_front = self.get_relative(cur_node_coords[0],cur_node_coords[1],new_dir_1)[2]
            turning_2_tile_in_front = self.get_relative(cur_node_coords[0],cur_node_coords[1],new_dir_2)[2]

            if turning_1_tile_in_front != '#':
                neighbours.add(((cur_node_coords),new_dir_1))
            
            if turning_2_tile_in_front != '#':
                neighbours.add(((cur_node_coords),new_dir_2))

            # For stepping forward, need to make sure that we don't step into a wall
            step_forward_tile = self.get_relative(cur_node_coords[0],cur_node_coords[1],cur_node_dir)

            if step_forward_tile[2] != '#':
                neighbours.add(((step_forward_tile[0],step_forward_tile[1]), cur_node_dir))
            
            for neighbour in neighbours:
                if cur_node_dir == neighbour[1]:
                    node_to_neighbour_cost = 1
                else:
                    node_to_neighbour_cost = 1000

                cost = costs[cur_node[1]] + node_to_neighbour_cost 

                if cost <= costs[neighbour]:
                    prevs[neighbour] = cur_node[1]
                    costs[neighbour] = cost
                    nodes_to_visit.put((cost, neighbour))

                    if neighbour[0] == self.end:
                        path_str = ''
                        cur_tile = neighbour
                        while(prevs[cur_tile]):
                            prev_tile, prev_dir = prevs[cur_tile]
                            path_str += prev_dir
                            cur_tile = (prev_tile,prev_dir)
                        routes.add((path_str[::-1], costs[neighbour]))

        
        
        
        return routes

            

In [142]:
maze_map = input.split('\n')

maze = Maze(maze_map)
#maze.print_grid()

best_paths = maze.find_lowest_scoring_path()
#maze.draw_path(best_paths)
best_paths

{('>>>>>^^^^^<<<<<^^^^^^^^^>>>^^^^^>>>vvv>>>^^^^^<<<<<<<^^^>>>^^^<<<^^^^^^^^^>>>>>^^^<<<<<^^^>>>>>>>>>^^^^^^^>>>vvv>>>^^^^^<<<^^^>>>^^^>>>^^^<<<<<^^^<<<<<<<<<<<^^^>>>^^^<<<^^^>>>^^^<<<^^^>>>>>>>>>^^^^^<<<<<<<<<^^^>>>^^^<<<^^^^^^^^^>>>vvv>>>>>>>^^^<<<<<^^^^^^^<<<<<^^^^^^^^^>>>vvv>>>>>>>^^^<<<<<^^^<<<^^^^^<<<^^^^^>>>^^^<<<^^^>>>^^^^^^^<<<^^^^^^^^^>>>>>>>>>^^^^^<<<^^^^^^^^^^^^^<<<^^^>>>^^^>>>vvvvv>>>>>>>>>>>>>>>>>>>>>>>>>vvv>>>>>>>^^^^^>>>>>>>>>>>>>>>>>>>>>^^^>>>>>>>>>>>>>>>>>>>vvvvv>>>>>vvv>>>^^^>>>>>vvv>>>>>>>^^^>>>vvvvv>>>>>^^^^^^^^^>>>vvvvv>>>>>>>^^^^^>>>>>vvv>>>^^^>>>vvvvv>>>vvv>>>>>^^^^^^^>>>>>>>>>>>>>>>>>>',
  122492),
 ('>>>>>^^^^^<<<<<^^^^^^^^^>>>^^^^^>>>vvv>>>^^^^^<<<<<<<^^^>>>^^^<<<^^^^^^^^^>>>>>^^^<<<<<^^^>>>>>>>>>^^^^^^^>>>vvv>>>^^^^^<<<^^^>>>^^^>>>^^^<<<<<^^^<<<<<<<<<<<^^^>>>^^^<<<^^^>>>^^^<<<^^^>>>>>>>>>^^^^^<<<<<<<<<^^^>>>^^^<<<^^^^^^^^^>>>vvv>>>>>>>^^^<<<<<^^^^^^^<<<<<^^^^^^^^^>>>vvv>>>>>>>^^^<<<<<^^^<<<^^^^^<<<^^^^^>>>^^^<<<^^^>>>^^^^^^^<<<^^^^^^^^^>>>>>>>>>^^^^^<<<^^^^^

In [143]:
maze = Maze(maze_map)

In [144]:
all_occupied = set()

for path in best_paths:
    if len(path[0]) == 1:
        continue
    all_occupied = all_occupied.union(maze.draw_path(path[0], return_set_occupied=True))

In [145]:
len(all_occupied)

520